## MA-GRYP Task

Labeling the candlestick combination of R,G,Y and Price from top to bottom

PRGY refering to Price(Close Price), MovingAverage200(R), MovingAverage100(G), MovingAverage14(Y)

Ratio between these 4 lines also labeled in %

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
# from zigzag import *
from tqdm import tqdm
import sys

In [2]:

data = pd.read_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\data\\Data_OANDA\\USD_CAD_M15_2005_202206.csv', encoding='utf-8',header=0)
#data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%dT%H:%M:%S.000000000Z')
data.reset_index(drop=True, inplace=True)

In [ ]:
# done: "AUD_USD_M15_2005_202206", 'EUR_USD_M15_2005_202206',

FILE_NAME_LIST = [
'GBP_USD_M15_2005_202206',
'USD_CAD_M15_2005_202206','USD_JPY_M15_2005_202206','USD_CHF_M15_2005_202206',
'NZD_USD_M15_2005_202206','AUD_CAD_M15_2005_202206'
]

#change directroy
data_ls = []
for i in FILE_NAME_LIST:
    file = pd.read_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\data\\Data_OANDA\\{i}.csv', encoding='utf-8',header=0)
    file.reset_index(drop=True, inplace=True)
    file['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%dT%H:%M:%S.000000000Z')
    data_ls.append(file)

In [43]:
data  = data[250:500]
# moving average 200
MA200 = data['close'].rolling(window =200).mean()
# moving average 100
MA100 = data['close'].rolling(window =100).mean()
# moving average 14
MA14 = data['close'].rolling(window =14).mean()

fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])

# fig.layout = dict(xaxis=dict(type="category"))
fig.add_trace(go.Scatter(x=data.index, y= MA200, mode='lines', line=dict(width = 1),marker_color = 'red',
                        showlegend=True, name = 'MovingAverage200'))

fig.add_trace(go.Scatter(x=data.index, y= MA100, mode='lines', line=dict(width = 1),marker_color = 'green', 
                        showlegend=True, name = 'MovingAverage100'))

fig.add_trace(go.Scatter(x=data.index, y= MA14,mode='lines', line=dict(width = 1),marker_color = 'yellow',
                        showlegend=True, name = 'MovingAverage14'))

layout = go.Layout(
    plot_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=15,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    )
)
fig.update_layout(layout)

fig.show()

In [5]:
def compare(Price,Red,Green,Yellow):

    # This function use built-in function sort to speed up the sort process.
    # Sort these 4 values (MA200,MA100,MA14,Close) in order to correctly label them from top to bottom

    #price:close price, Red = MovingAvg200, Green = MovingAvg100, Yellow = MovingAvg14
    price_list = {'Price':Price, 'Red':Red, 'Green':Green,'Yellow':Yellow}
    # sort the dictionary base on the dict.values
    sorted_price_list = (sorted(price_list.items(), key=lambda x: x[1],reverse =True))
    # eg sorted_price_list dict(("Price":196.820)  ('Red':196.820) ('Yellow':196.780) ('Green':196.750))
    label = ''
    for i in sorted_price_list:
        # use dict keys and the first capital letter for labelling
        label+=i[0][0]
    # print(sorted_price_list,label)
    return label

def distance(Price,Red,Green,Yellow):

    # This function calculate the distance (%) between 4 lines (MA200,MA100,MA14,Close)
    # use the sort function to sort them base on the dict.values  return (max, second, third, fourth)
    # then calculate the difference between them and divide by the range so the percentage values add up to 100%   

    #price:close price, Red = MovingAvg200, Green = MovingAvg100, Yellow = MovingAvg14
    price_list = {'Price':Price, 'Red':Red, 'Green':Green,'Yellow':Yellow}
    # sort the dictionary base on the dict.values
    sorted_price_list = (sorted(price_list.items(), key=lambda x: x[1],reverse =True))
    # distance between the maximum and minimum
    range = sorted_price_list[0][1] - sorted_price_list[3][1]
    # distance between the maximum and the second (%)
    ratio_top = (sorted_price_list[0][1] - sorted_price_list[1][1]) / range 
    # distance between the second and the third (%)
    ratio_mid = (sorted_price_list[1][1] - sorted_price_list[2][1]) / range
    # distance between the third and the fourth (%)
    ratio_bottom = (sorted_price_list[2][1] - sorted_price_list[3][1]) / range

    return ratio_top,ratio_mid,ratio_bottom



def MA_PRGY_Task(data):  
    index = 0
    # moving average 200
    MA200 = data['close'].rolling(window =200).mean()
    # moving average 100
    MA100 = data['close'].rolling(window =100).mean()
    # moving average 14
    MA14 = data['close'].rolling(window =14).mean()

    # add 4 columns to the dataframe
    data['event'] = ''
    data['ratio top'] = ''
    data['ratio mid'] = ''
    data['ratio bottom'] = ''

    label_list = []
    for i in tqdm(range(200,len(data),1)):
        Price = data.close[i]
        Red = MA200[i]
        Green = MA100[i]
        Yellow = MA14[i]
        # sort them in order return the label
        label = compare(Price,Red,Green,Yellow)

        # calculate the distance ratio
        ratio_top,ratio_mid,ratio_bottom = distance(Price,Red,Green,Yellow)
        data['ratio top'][i] = ratio_top
        data['ratio mid'][i] = ratio_mid
        data['ratio bottom'][i] = ratio_bottom
        
        # first label append to the list start at 200 
        if len(label_list) == 0:
            data['event'][200] = label
        label_list.append(label)

        if len(label_list) > 1:
            # label added if is different from previou one
            if label_list[-1] != label_list[-2] :
                data['event'][i] = label
             # label not be added if is same from previou one
            elif label_list[-1] == label_list[-2] :
                data['event'][i] = ''

    print(f'output excel file: USD_CAD_M15_2005_202206')
    data.to_excel(f'C:\\Users\\HFY\\Desktop\\capstone\\w9\\USD_CAD_M15_2005_202206_events.xlsx',index=False)
    print(f'Finish index: USD_CAD_M15_2005_202206')

In [6]:
MA_PRGY_Task(data)

100%|██████████| 439927/439927 [1:09:28<00:00, 105.53it/s]


output excel file: USD_CAD_M15_2005_202206
Finish index: USD_CAD_M15_2005_202206


In [ ]:
# # price:close price, Red = MovingAvg200, Green = MovingAvg100, Yellow = MovingAvg14
# # compare(Price, Red, Green, Yellow)
# compare(196,195,194,193)

# # if all the parameters are equal (output base on the input order)
# compare(196,196,196,196)
# # output:[('Price', 2), ('Red', 2), ('Green', 2), ('Yellow', 2)]  PRGY

# # two of them are equal (same values return order also base on the input order)
# compare(1,2,2,1)
# # output: [('Red', 2), ('Green', 2), ('Price', 1), ('Yellow', 1)] RGPY

# #three of them are equal (same values return order also base on the input order)
# compare(2,1,2,2)
# # output: [('Price', 2), ('Green', 2), ('Yellow', 2), ('Red', 1)] PGYR

# # if the input values are the same, it will return the order base on the input sequence.

In [ ]:
# Test
data  = data[0:1000]
MA_PRGY_Task(data,0)
# output result
result = dict(data.event.value_counts())
for idx,values in result.items():
    if idx != '':
        print(f'{idx} {values} times')

RGYP 54 times
RGPY 50 times
PYGR 15 times
GRYP 13 times
YPGR 12 times
GRPY 12 times
GPRY 10 times
RPGY 10 times
PYRG 9 times
YGPR 7 times
GYRP 7 times
YPRG 6 times
YRPG 6 times
GYPR 5 times
RYGP 4 times
RPYG 3 times
PGYR 2 times
GPYR 2 times
YRGP 2 times
RYPG 2 times
YGRP 1 times
PRYG 1 times
PGRY 1 times


## Trading Strategy Part

Golden cross

Def: A bullish signal generated when the 50-day(short-term) moving average crosses above the 200-day(long-term) moving average

here used moving average 14 Y as short term moving average, and MA100 G & MA200 R as long term moving average.

***When Y is above the other two moving average consider as bullish signal and enter the market.***

***Exist when the current price is above or below the profit taken and stop loss level.***


In [4]:
labeled_data = pd.read_excel('./excel/USD_CAD_M15_2005_202206_events.xlsx')

In [9]:
# show result
result = dict(labeled_data.event.value_counts())
for idx,values in result.items():
    if idx != '':
        print(f'{idx} {values} times')

RGPY 13244 times
YPGR 12596 times
RGYP 12563 times
PYGR 11861 times
GRYP 4428 times
PYRG 4243 times
RPGY 4200 times
RPYG 4109 times
GRPY 4028 times
YGPR 3980 times
GYPR 3929 times
YPRG 3789 times
RYPG 3434 times
GPYR 3321 times
RYGP 2691 times
GYRP 2578 times
PRYG 2569 times
PGYR 2485 times
GPRY 1583 times
YRPG 1447 times
PRGY 1392 times
YGRP 1302 times
YRGP 905 times
PGRY 903 times


In [ ]:
def trade_strategy(data, stop_loss, stop_profit):

    pass

In [10]:
data = labeled_data.copy()
data.fillna('',inplace = True)

## USD_CAD_M15_2005_202206_trade

stop_loss = 0.0015

profit_taken = 0.0008

In [5]:
data = labeled_data.copy()
data.fillna('',inplace = True)
data['action'] = ''
data['P/L'] = ''
data['Total Profit'] = ''
stop_loss =  0.0001 * 15
profit_taken = 0.0001 * 8
row = 0
total_profit = 0
is_trading = False
wining_trade, losing_trade = 0,0
for i in (data.iterrows()):
    
    current_price = i[1]['close']

    #print(current_price)
    # stop loss condition
    if is_trading and (current_price < (enter_price * (1-stop_loss)))  :
        data['action'][row] = 'sell'
        data['P/L'][row] =  current_price - enter_price
        is_trading = False
        trade_gain = (current_price - enter_price)
        print(f'Stop Loss: {trade_gain}',end = "\r")
        total_profit += trade_gain
        losing_trade+=1
    # profit taken condition
    if is_trading and (current_price > (enter_price * (1+profit_taken))):
        data['action'][row] = 'sell'
        data['P/L'][row] =  current_price - enter_price
        is_trading = False
        trade_gain = (current_price - enter_price)
        print(f'Profit gain: {trade_gain}',end = "\r")
        total_profit += trade_gain
        wining_trade+=1

    # hold condition
    elif is_trading:
        data['action'][row] = 'hold'
        data['P/L'][row] =  current_price - enter_price

    if not(is_trading) and len(i[1]['event']) > 0:
        if i[1]['event'][0] == 'Y':
            data['action'][row] = 'buy'
            enter_price = i[1]['close']
            data['P/L'][row] =  current_price - enter_price
            is_trading = True
    
    data['Total Profit'][row] = total_profit
    row+=1
    
print(f'Total Profit {total_profit}')
print(f'wining trades: {wining_trade} \nlosing trades: {losing_trade} \nwin rate {wining_trade/(wining_trade+losing_trade) *100:.2f}%' )

Total Profit 0.6717600000000208479
wining trades: 6863 
losing trades: 3991 
win rate 63.23%


In [30]:
data.to_excel('./excel/USD_CAD_M15_2005_202206_trade.xlsx' ,index = False)

## USD_CAD_M15_2005_202206_trade

stop_loss = 0.03

profit_taken = 0.02

In [33]:
data = labeled_data.copy()
data.fillna('',inplace = True)
data['action'] = ''
data['P/L'] = ''
data['Total Profit'] = ''
stop_loss = 0.03
profit_taken = 0.02
row = 0
total_profit = 0
is_trading = False
wining_trade, losing_trade = 0,0
for i in (data.iterrows()):
    
    current_price = i[1]['close']
    #print(current_price)
    # stop loss condition
    if is_trading and (current_price < (enter_price * (1-stop_loss))) :
        data['action'][row] = 'sell'
        data['P/L'][row] =  current_price - enter_price
        is_trading = False
        trade_gain = (current_price - enter_price)
        print(f'Stop Loss: {trade_gain}',end = "\r")
        total_profit += trade_gain
        losing_trade+=1
    # profit taken condition
    if is_trading and (current_price > (enter_price * (1+profit_taken))):
        data['action'][row] = 'sell'
        data['P/L'][row] =  current_price - enter_price
        is_trading = False
        trade_gain = (current_price - enter_price)
        print(f'Profit gain: {trade_gain}',end = "\r")
        total_profit += trade_gain
        wining_trade+=1

    # hold condition
    elif is_trading:
        data['action'][row] = 'hold'
        data['P/L'][row] =  current_price - enter_price

    if not(is_trading) and len(i[1]['event']) > 0:
        if i[1]['event'][0] == 'Y':
            data['action'][row] = 'buy'
            enter_price = i[1]['close']
            data['P/L'][row] =  current_price - enter_price
            is_trading = True
    
    
    data['Total Profit'][row] = total_profit
    row+=1
    
print(f'Total Profit {total_profit}')
print(f'wining trades: {wining_trade} \nlosing trades: {losing_trade} \nwin rate {wining_trade/(wining_trade+losing_trade) *100:.2f}%' )

Total Profit 0.083079999999998646
wining trades: 132 
losing trades: 85 
win rate 60.83%


In [34]:
data.to_excel('./excel/USD_CAD_M15_2005_202206_trade2.xlsx' ,index = False)

## USD_CAD_M15_2005_202206_trade

sl = 0.02, pt = 0.02, ratio = 1:1

In [35]:
data = labeled_data.copy()
data.fillna('',inplace = True)
data['action'] = ''
data['P/L'] = ''
data['Total Profit'] = ''
stop_loss = 0.02
profit_taken = 0.02
row = 0
total_profit = 0
is_trading = False
wining_trade, losing_trade = 0,0
for i in (data.iterrows()):
    
    current_price = i[1]['close']
    #print(current_price)
    # stop loss condition
    if is_trading and (current_price < (enter_price * (1-stop_loss))) :
        data['action'][row] = 'sell'
        data['P/L'][row] =  current_price - enter_price
        is_trading = False
        trade_gain = (current_price - enter_price)
        print(f'Stop Loss: {trade_gain}',end = "\r")
        total_profit += trade_gain
        losing_trade+=1
    # profit taken condition
    if is_trading and (current_price > (enter_price * (1+profit_taken))):
        data['action'][row] = 'sell'
        data['P/L'][row] =  current_price - enter_price
        is_trading = False
        trade_gain = (current_price - enter_price)
        print(f'Profit gain: {trade_gain}',end = "\r")
        total_profit += trade_gain
        wining_trade+=1

    # hold condition
    elif is_trading:
        data['action'][row] = 'hold'
        data['P/L'][row] =  current_price - enter_price

    if not(is_trading) and len(i[1]['event']) > 0:
        if i[1]['event'][0] == 'Y':
            data['action'][row] = 'buy'
            enter_price = i[1]['close']
            data['P/L'][row] =  current_price - enter_price
            is_trading = True
    
    data['Total Profit'][row] = total_profit
    row+=1
    
print(f'Total Profit {total_profit}')
print(f'wining trades: {wining_trade} \nlosing trades: {losing_trade} \nwin rate {wining_trade/(wining_trade+losing_trade) *100:.2f}%' )

Total Profit 0.126069999999997687
wining trades: 154 
losing trades: 146 
win rate 51.33%


In [40]:
# data.to_excel('./excel/USD_CAD_M15_2005_202206_trade3.xlsx' ,index = False)

## GBP_USD_M15_2005_202206

stop_loss = 0.02
profit_taken = 0.02

ratio = 1:1

In [2]:
labeled_data = pd.read_excel('./excel/GBP_USD_M15_2005_202206_trade.xlsx')

In [49]:
data = labeled_data.copy()
data.event.value_counts()

RGPY    13256
YPGR    13067
RGYP    12536
PYGR    12310
PYRG     4086
GYPR     4064
RPYG     4054
GRYP     3982
YGPR     3904
RPGY     3863
GRPY     3631
YPRG     3625
GPYR     3424
RYPG     3365
RYGP     2489
PGYR     2446
PRYG     2442
GYRP     2373
GPRY     1307
YRPG     1236
PRGY     1185
YGRP     1161
YRGP      750
PGRY      736
Name: event, dtype: int64

In [50]:
data = labeled_data.copy()
data.fillna('',inplace = True)
data['action'] = ''
data['P/L'] = ''
data['Total Profit'] = ''
stop_loss = 0.02
profit_taken = 0.02
row = 0
total_profit = 0
is_trading = False
wining_trade, losing_trade = 0,0
for i in (data.iterrows()):
    
    current_price = i[1]['close']
    #print(current_price)
    
    if not(is_trading) and len(i[1]['event']) > 0:
        if i[1]['event'][0] == 'Y':
            data['action'][row] = 'buy'
            enter_price = i[1]['close']
            data['P/L'][row] =  current_price - enter_price
            is_trading = True

    # stop loss condition
    if is_trading and (current_price < (enter_price * (1-stop_loss))) :
        data['action'][row] = 'sell'
        data['P/L'][row] =  current_price - enter_price
        is_trading = False
        trade_gain = (current_price - enter_price)
        print(f'Stop Loss: {trade_gain}',end = "\r")
        total_profit += trade_gain
        losing_trade+=1
    # profit taken condition
    if is_trading and (current_price > (enter_price * (1+profit_taken))):
        data['action'][row] = 'sell'
        data['P/L'][row] =  current_price - enter_price
        is_trading = False
        trade_gain = (current_price - enter_price)
        print(f'Profit gain: {trade_gain}',end = "\r")
        total_profit += trade_gain
        wining_trade+=1
    # hold condition
    elif is_trading:
        data['action'][row] = 'hold'
        data['P/L'][row] =  current_price - enter_price
    
    data['Total Profit'][row] = total_profit
    row+=1

print(f'Total Profit {total_profit}')
print(f'wining trades: {wining_trade} \nlosing trades: {losing_trade} \nwin rate {wining_trade/(wining_trade+losing_trade) *100:.2f}%' )

Total Profit 0.215159999999998247
wining trades: 160 
losing trades: 151 
win rate 51.45%


In [38]:
data.to_excel('./excel/GBP_USD_M15_2005_202206_trade1.xlsx' ,index = False)

## GBP_USD_M15_2005_202206

stop_loss = 0.15
profit_taken = 0.08

ratio = 0.53

In [3]:
data = labeled_data.copy()
data.fillna('',inplace = True)
data['action'] = ''
data['P/L'] = ''
data['Total Profit'] = ''
stop_loss = 0.15
profit_taken = 0.08
row = 0
total_profit = 0
is_trading = False
wining_trade, losing_trade = 0,0
for i in (data.iterrows()):
    
    current_price = i[1]['close']
    #print(current_price)
    
    if not(is_trading) and len(i[1]['event']) > 0:
        if i[1]['event'][0] == 'Y':
            data['action'][row] = 'buy'
            enter_price = i[1]['close']
            data['P/L'][row] =  current_price - enter_price
            is_trading = True

    # stop loss condition
    if is_trading and (current_price < (enter_price * (1-stop_loss))) :
        data['action'][row] = 'sell'
        data['P/L'][row] =  current_price - enter_price
        is_trading = False
        trade_gain = (current_price - enter_price)
        print(f'Stop Loss: {trade_gain}',end = "\r")
        total_profit += trade_gain
        losing_trade+=1
    # profit taken condition
    if is_trading and (current_price > (enter_price * (1+profit_taken))):
        data['action'][row] = 'sell'
        data['P/L'][row] =  current_price - enter_price
        is_trading = False
        trade_gain = (current_price - enter_price)
        print(f'Profit gain: {trade_gain}',end = "\r")
        total_profit += trade_gain
        wining_trade+=1
    # hold condition
    elif is_trading:
        data['action'][row] = 'hold'
        data['P/L'][row] =  current_price - enter_price
    
    data['Total Profit'][row] = total_profit
    row+=1

print(f'Total Profit {total_profit}')
print(f'wining trades: {wining_trade} \nlosing trades: {losing_trade} \nwin rate {wining_trade/(wining_trade+losing_trade) *100:.2f}%' )

Total Profit -0.6938100000000003
wining trades: 7 
losing trades: 6 
win rate 53.85%


In [ ]:
data.to_excel('./excel/GBP_USD_M15_2005_202206_trade2.xlsx' ,index = False)